In [6]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [9]:
!pip3 install --upgrade rank_bm25
!pip3 install --upgrade nltk
!pip3 install gensim==3.8.3

You should consider upgrading via the 'c:\python39\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\python39\python.exe -m pip install --upgrade pip' command.


Using legacy 'setup.py install' for gensim, since package 'wheel' is not installed.
    Running setup.py install for gensim: started
    Running setup.py install for gensim: finished with status 'done'


You should consider upgrading via the 'c:\python39\python.exe -m pip install --upgrade pip' command.


In [10]:
from nltk.tokenize import word_tokenize
import pickle, json
import pandas as pd

from rank_bm25 import *
from gensim.parsing.preprocessing import STOPWORDS
from gensim.parsing.preprocessing import remove_stopwords
import re

In [11]:
def stopwords_remove(lst):
    lst1=list()
    for str in lst:
        text_tokens = word_tokenize(str)
        tokens_without_sw = [word for word in text_tokens if not word in all_stopwords_gensim]
        str_t = " ".join(tokens_without_sw)
        lst1.append(str_t)
 
    return lst1

In [12]:
def spl_chars_removal(lst):
    lst1=list()
    for element in lst:
        str= ""
        str = re.sub("[^A-Za-z0-9]+"," ",element)
        lst1.append(str)
    return lst1

In [13]:
def remove_non_ascii(sentence):
    temp_str = sentence.encode('ascii', errors='ignore').decode("utf-8")
    result = " ".join(temp_str.split())
    return result

In [14]:
import nltk
from nltk.stem import PorterStemmer
ps = PorterStemmer() 
sentence = "Charles IV, King of Bohemia and Holy Roman Emperor, had a long and successful reign. The Empire he ruled from Prague expanded, and his subjects lived in peace and prosperity."
for word in sentence.split():
    print(ps.stem(word))
    
def sentence_stemmer(sentence):
    return [ps.stem(word) for word in sentence]


charl
iv,
king
of
bohemia
and
holi
roman
emperor,
had
a
long
and
success
reign.
the
empir
he
rule
from
pragu
expanded,
and
hi
subject
live
in
peac
and
prosperity.


In [15]:
with open('master_list_relabeled.json', 'r') as fp:
    master_list = json.load(fp)
    
job_desc_docs = []
for i in master_list:
    job_desc_docs.append(master_list[i]['job_description'])

print(len(job_desc_docs))
job_desc_docs_raw = job_desc_docs

job_desc_docs = [remove_stopwords(i).split() for i in job_desc_docs]
job_desc_docs = [spl_chars_removal(i) for i in job_desc_docs]
job_desc_docs_stemmed = [sentence_stemmer(i) for i in job_desc_docs]
print(job_desc_docs_stemmed[1])

with open('job_desc_tokenized.pickle', 'wb') as handle:
    pickle.dump(job_desc_docs_stemmed, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print("saved")

7426
['job', 'descript', 'the', 'success', 'candid', 'attach', 'branch', 'offic', 'perform', 'follow', 'duti', 'administr', 'assistant ', 'describ', 'duties ', 'provid', 'gener', 'administr', 'support', 'office ', 'data', 'entri', 'file', 'document', ' hardcopi', 'electronic ', 'prepar', 'invoices ', 'correspond', 'support', 'documents ', 'handl', 'daili', 'routin', 'invoic', 'relat', 'duties ', 'commun', 'custom', 'work', 'partners ', 'undertak', 'duti', 'assign', 'superior', 'time', 'time ', 'e', 'xpectations ', 'meticul', 'abl', 'multi task', 'profici', 'ms', 'offic', 'skill', 'basic', 'use', 'sap', 'system ', ' don t', 'worri', 'guid', 'you ', 'possess', 'good', 'interperson', 'commun', 'skill', ' we', 'need', 'happi', 'people ', 'exhibit', 'good', 'work', 'attitud', 'team ori', 'spirit', 'career', 'level ', 'junior', 'execut', 'qualification ', 'profession', 'certificate nitec ', 'diploma ', 'advanced higher gradu', 'diploma', 'year', 'experience ', '1 2', 'year', 'job', 'type ', 

In [18]:
import pandas as pd
resume_raw = pd.read_csv('resume_clean_dataset.csv')
resume_list = []

for resume in resume_raw['Resume']:
    resume_list.append(resume)
    
print(len(resume_list))
resume_list_raw = resume_list

resume_list = [remove_non_ascii(i) for i in resume_list]
resume_list = [remove_stopwords(i).split() for i in resume_list]
resume_list = [spl_chars_removal(i) for i in resume_list]
resume_list_stemmed = [sentence_stemmer(i) for i in resume_list]
    
print(resume_list_stemmed[0])
with open('resume_list_stemmed.pickle', 'wb') as handle:
    pickle.dump(resume_list_stemmed, handle, protocol=pickle.HIGHEST_PROTOCOL)


166
['skill', ' ', 'program', 'languages ', 'python', ' pandas ', 'numpy ', 'scipy ', 'scikit learn ', 'matplotlib ', 'sql ', 'java ', 'javascript jquery ', ' ', 'machin', 'learning ', 'regression ', 'svm ', 'nave', 'bayes ', 'knn ', 'random', 'forest ', 'decis', 'trees ', 'boost', 'techniques ', 'cluster', 'analysis ', 'word', 'embedding ', 'sentiment', 'analysis ', 'natur', 'languag', 'processing ', 'dimension', 'reduction ', 'topic', 'model', ' lda ', 'nmf ', 'pca', ' ', 'neural', 'nets ', ' ', 'databas', 'visualizations ', 'mysql ', 'sqlserver ', 'cassandra ', 'hbase ', 'elasticsearch', 'd3 js ', 'dc js ', 'plotly ', 'kibana ', 'matplotlib ', 'ggplot ', 'tableau ', ' ', 'others ', 'regular', 'expression ', 'html ', 'css ', 'angular', '6 ', 'logstash ', 'kafka ', 'python', 'flask ', 'git ', 'docker ', 'vision', ' ', 'open', 'cv', 'understand', 'deep', 'learning educ', 'detail', 'data', 'scienc', 'assur', 'associ', 'data', 'scienc', 'assur', 'associ', ' ', 'ernst', ' ', 'young', 'llp

In [23]:
tokenized_corpus = job_desc_docs_stemmed
bm25 = BM25Okapi(tokenized_corpus)

df_best = pd.DataFrame(columns = ["resume", "job"])
df_worst = pd.DataFrame(columns = ["resume", "job"])

# for idx in range(250,280):
for idx in range(len(resume_list_stemmed)):
    tokenized_query = resume_list_stemmed[idx]
# print("---------------------Resume-----------------------")
# print(resume_list_raw[idx])
# print("---------------------JD-----------------------")

#     doc_top = bm25.get_top_n(tokenized_query, job_desc_docs_raw, n=5)
    # print(doc_top[0])
    doc_scores = bm25.get_scores(tokenized_query)
#     doc_scores_dict = {key: value for (key, value) in enumerate(doc_scores)}
    top_matches = sorted(range(len(doc_scores)), key=lambda i: doc_scores[i])[-10:]
    worst_matches = sorted(range(len(doc_scores)), key=lambda i: doc_scores[i])[0:5]
    print(top_matches)
    
    counter = 0
#     for doc_idx, score in enumerate(doc_scores):
#         if score == max(doc_scores):
#             print(idx, doc_idx)
    for match in top_matches:
        if doc_scores[match] > 200:
            df_best = df_best.append({"resume": remove_non_ascii(resume_list_raw[idx]), "job": job_desc_docs_raw[match]}, ignore_index=True)
    
    for match in worst_matches:
        df_worst = df_worst.append({"resume": remove_non_ascii(resume_list_raw[idx]), "job": job_desc_docs_raw[match]}, ignore_index=True)
            

df_best.to_csv('matches_best.tsv', sep = '\t')
df_worst.to_csv('matches_worst.tsv', sep = '\t')

[4173, 6243, 4296, 5580, 6199, 4482, 4236, 6193, 3238, 2400]
[1586, 1796, 7107, 2868, 6193, 1991, 5592, 6546, 2701, 2135]
[6791, 1228, 6546, 3235, 2959, 5592, 6176, 4613, 2135, 2400]
[4296, 5580, 4803, 2400, 6414, 4336, 4260, 4173, 4482, 4236]
[5592, 4709, 2434, 4297, 2033, 765, 2701, 1991, 6546, 2135]
[232, 4709, 2868, 4412, 5937, 6546, 5580, 2701, 1991, 2135]
[2400, 3238, 6243, 4236, 6199, 5580, 6414, 4296, 6193, 4482]
[7212, 6629, 6755, 6763, 5580, 6768, 5560, 5078, 2400, 6193]
[5580, 4296, 5912, 6193, 6243, 6763, 5713, 7212, 6768, 5078]
[6453, 5630, 4236, 6366, 4296, 6243, 6414, 4271, 5580, 4482]
[296, 2290, 6711, 3685, 613, 5534, 3631, 120, 6091, 5057]
[679, 482, 2693, 3963, 5403, 5119, 25, 7042, 4808, 3602]
[2533, 1482, 184, 1916, 2154, 2343, 3166, 7240, 5119, 5697]
[184, 792, 2179, 3028, 2921, 1482, 5119, 3427, 3166, 3234]
[1656, 6186, 3645, 279, 1266, 3383, 2631, 5697, 4062, 3166]
[7240, 3645, 2179, 3049, 1266, 3028, 184, 5119, 1482, 3166]
[5857, 2319, 5693, 7380, 2631, 5752, 5

In [ ]:
print(master_list[str(6259)])